<a href="https://colab.research.google.com/github/Tiabet/NLP/blob/master/Review_classification_4types_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers accelerate sentencepiece
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machi

In [3]:
from tqdm import tqdm

In [4]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(
    model,
    use_auth_token=True,
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
import pandas as pd

In [6]:
df_train = pd.read_csv("drive/MyDrive/train_data.csv")
df_test =   pd.read_csv("drive/MyDrive/test_data.csv")

In [7]:
def gen(x, max_length=300):
    sequences = pipeline(
        x,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )

    return sequences[0]["generated_text"].replace(x, "")

In [9]:
preds = []

for idx in tqdm(range(len(df_test))):
        row = df_test.iloc[idx]

        random_1 = df_train[df_train['label']==1].sample(1)
        random_2 = df_train[df_train['label']==2].sample(1)
        random_3 = df_train[df_train['label']==3].sample(1)
        random_4 = df_train[df_train['label']==4].sample(1)

        example_sentence = f"""
        주어진 리뷰가 긍정적인 리뷰면 1, 부정적인 리뷰면 2, 무의미한 리뷰면 3, 악의적인 리뷰면 4를 반환하세요.

        # 예시
        리뷰1 : {random_1['sentences'].values[0]} -> 1
        리뷰2 : {random_2['sentences'].values[0]} -> 2
        리뷰3 : {random_3['sentences'].values[0]} -> 3
        리뷰4 : {random_4['sentences'].values[0]} -> 4

        # 문제
        문장 : {row['sentences']}  ->
        """

        preds.append(gen(example_sentence, 1000))






 10%|█         | 2/20 [00:53<08:54, 29.68s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 1228, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
 50%|█████     | 10/20 [02:29<02:03, 12.34s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 20/20 [07:32<00:00, 22.62s/it]


In [46]:
preds

['예상 답변 : 1\n\n        # 설명\n        이 문장은 양 진짜 푸짐하고 너어무 맛있어요라는 긍정적인 리뷰입니다. 따라서 답변을 1로 반환합니다.',
 '\n        # 풀이\n        1. 리뷰1 : 존맛탱 ㅠ 역시 맛나용 -> 1\n        2. 리뷰2 : 고기도 뻣뻣하고 고기가 불리되어 맛이 없고 둒껍고 계란찜도 엉망으로 해오고 먹으란건지 의문 -> 2\n        3. 리뷰3 : 맛나요~~ 맛나요~~ 맛나요~~ -> 3\n        4. 리뷰4 : 맛이 정말 심각해요 -> 4\n        # 설명\n        1. 리뷰1 : 존맛탱 ㅠ 역시 맛나용 -> 1 : 이 리뷰는 긍정적인 리뷰임을 나타내는 1의 값을 반환합니다.\n        2. 리뷰2 : 고기도 뻣뻣하고 고기가 불리되어 맛이 없고 둒껍고 계란찜도 엉망으로 해오고 먹으란건지 의문 -> 2 : 이 리뷰는 부정적인 리뷰임을 나타내는 2의 값을 반환합니다.\n        3. 리뷰3 : 맛나요~~ 맛나요~~ 맛나요~~ -> 3 : 이 리뷰는 무의미한 리뷰임을 나타내는 3의 값을 반환합니다.\n        4. 리뷰4 : 맛이 정말 심각해요 -> 4 : 이 리뷰는 악의적인 리뷰임을 나타내는 4의 값을 반환합니다.\n    """\n\n    def __init__(self, review):\n        self.review = review\n\n    def get_rating(self):\n        if self.',
 '\n',
 '예상결과 : 1\n    """\n\n    def __init__(self, sentence):\n        self.sentence = sentence\n\n    def get_rating(self):\n        # 1. 긍정적인 리뷰\n        if self.sentence.startswith("맵기도"):\n            return 1\n        # 2. 부정적인 리

In [45]:
df_test.loc[1]

sentences    맛잇개먹엇어요. ㅋㅋㅋㅋ
label                    3
Name: 1, dtype: object

In [47]:
df_test['label']

0     1
1     3
2     1
3     1
4     1
5     3
6     1
7     3
8     1
9     1
10    1
11    3
12    3
13    1
14    2
15    1
16    1
17    2
18    1
19    1
Name: label, dtype: int64

In [50]:
pred_label = [1,2,2,1,1,4,1,3,1,1,1,1,1,1,1,1,1,2,1,1]

In [51]:
from sklearn.metrics import accuracy_score

accuracy_score(pred_label, df_test['label'])

0.7